<a href="https://colab.research.google.com/github/the-Soke/FAW-AI-MODEL/blob/Trained.faw.ipynb/Trained_faw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --quiet
!git clone https://github.com/the-Soke/FAW-AI-MODEL.git
%cd FAW-AI-MODEL

Cloning into 'FAW-AI-MODEL'...
remote: Enumerating objects: 747, done.
remote: Counting objects: 100% (747/747), done.
remote: Compressing objects: 100% (746/746), done.
remote: Total 747 (delta 4), reused 731 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (747/747), 42.43 MiB | 30.23 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/FAW-AI-MODEL


In [ ]:
!ls /content/FAW-AI-MODEL

 dataset
'Fall-Armyworm-Spodoptera-frugipe-landscape- soil background 2.zip'
 Fall-Armyworm-Spodoptera-frugipe-landscape.zip
'Fall Armyworm (Spodoptera frugiperda) - floor background.zip'
'Fall Armyworm (Spodoptera frugiperda) - Soil background 3.zip'
 Fall-Armyworm-Spodoptera-frugipe.zip
 FAW_AI_PROJECT.ipynb
 README.md


In [ ]:
!ls /content/FAW-AI-MODEL/dataset


damage	floor  soil


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define dataset path
data_dir = "/content/FAW-AI-MODEL/dataset"

# Define transforms (resize, normalize, etc.)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Load full dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Split into train and val sets (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("✅ Dataset loaded successfully!")
print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Classes:", full_dataset.classes)



✅ Dataset loaded successfully!
Train samples: 580
Validation samples: 145
Classes: ['damage', 'floor', 'soil']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
# Load pretrained ResNet18
model = models.resnet18(weights='IMAGENET1K_V1')

# Replace the last fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 classes

model = model.to(device)
print("✅ ResNet18 model ready for training!")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 89.3MB/s]


✅ ResNet18 model ready for training!


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 5
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    # Log results
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch [1/6] Train Loss: 0.2354, Train Acc: 92.24% | Val Loss: 9.2252, Val Acc: 34.48%
Epoch [2/6] Train Loss: 0.0347, Train Acc: 99.31% | Val Loss: 0.4458, Val Acc: 92.41%
Epoch [3/6] Train Loss: 0.0579, Train Acc: 97.76% | Val Loss: 9.5624, Val Acc: 16.55%
Epoch [4/6] Train Loss: 0.0174, Train Acc: 99.31% | Val Loss: 1.3349, Val Acc: 78.62%
Epoch [5/6] Train Loss: 0.0302, Train Acc: 98.45% | Val Loss: 0.0341, Val Acc: 99.31%
Epoch [6/6] Train Loss: 0.0363, Train Acc: 99.31% | Val Loss: 0.3407, Val Acc: 89.66%


In [10]:
# --- PHASE 2: Fine-tuning (unfreeze entire model) ---
for param in model.parameters():
    param.requires_grad = True  # <— Unfreeze everything

# Now use a smaller LR (so you don’t “destroy” the pretrained features)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 3
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    # Log results
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch [1/3] Train Loss: 0.0058, Train Acc: 100.00% | Val Loss: 0.0005, Val Acc: 100.00%
Epoch [2/3] Train Loss: 0.0078, Train Acc: 99.66% | Val Loss: 0.0028, Val Acc: 100.00%
Epoch [3/3] Train Loss: 0.0037, Train Acc: 100.00% | Val Loss: 0.0005, Val Acc: 100.00%


In [11]:
torch.save(model.state_dict(), "best_resnet18_faw.pth")
print("✅ Model saved successfully as best_resnet18_faw.pth")


✅ Model saved successfully as best_resnet18_faw.pth
